In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 무신사 인기검색어 300 크롤링

In [11]:
def hot300(date):
    
    url = "https://www.musinsa.com/ranking/keyword"
    
    # url 정보 불러오기
    response = requests.get(url)

    # html 형식으로 바꿔주기 
    html = BeautifulSoup(response.text, "html.parser")

    # df 변환을 위해 빈 리스트 만들기
    rank_list = []

    # 300개의 랭킹 크롤링
    for i in range(300):
        rank = html.select("p.p_srank")[i].text.replace(" ", "").split(".")[0]
        name = html.select("p.p_srank")[i].text.replace(" ", "").split(".")[1]

        if html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[2] == "":
            increase_cnt = html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[2].replace("", "0")
        else:
            increase_cnt = html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[2]

        if "▲" in html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[1]:
            increase = html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[1].replace("▲", "1")
        elif "▼" in html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[1]:
            increase = html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[1].replace("▼", "0")
        else:
            increase = html.select("p.p_srank_last")[i].text.replace(" ", "").split("\n")[1]

        rank_dict = {
            "rank": rank,
            "name": name,
            "increase": increase,
            "increase_cnt": increase_cnt
        }

        rank_list.append(rank_dict)
    
    # df 생성
    ranking = pd.DataFrame(rank_list, columns = ["rank", "name", "increase", "increase_cnt"])
    
    # 순위 타입 int로 변경
    ranking["rank"] = ranking["rank"].astype("int")
    
    # 증가 및 하락 column int로 변경
    ranking["increase_cnt"] = ranking["increase_cnt"].astype("int")
    
    # 저장 후 csv 내보내기
    ranking.to_csv(f"{date}_musinsa_top300.csv", index = False)
    
    return ranking

# 무신사 브랜드 TOP 100 크롤링

In [12]:
def brand100(date):
    
    url = "https://www.musinsa.com/ranking/brand"
    
    # url 정보 불러오기
    response = requests.get(url)

    # html 형식으로 바꿔주기 
    html = BeautifulSoup(response.text, "html.parser")
    
    # df 변환을 위해 빈 리스트 만들기
    brand_list = []
    url_list = []

    # Top 100까지만 존재하기 때문에 
    for i in range(100):
        name = html.select("p.brand_name")[i].text.strip()
        en_name = html.select("p.brand_name_en")[i].text.strip()

        brand_dict = {
            "name": name,
            "en_name": en_name
        }

        brand_list.append(brand_dict)

    for name in html.select('a.brandLogo'):
        url = name.attrs['href']

        url_dict = {
            "url": url
        }

        url_list.append(url_dict)

    url_df = pd.DataFrame(url_list)
    brand_df = pd.DataFrame(brand_list)

    total_brand = pd.concat([brand_df, url_df], axis = 1)
    total_brand = total_brand.reset_index()
    total_brand.columns = ["rank", "name", "en_name", "url"]

    total_brand.to_csv(f"{date}_musinsa_brand100.csv", index = False)
    
    return total_brand

# 무신사 Best item Top 100 크롤링

In [13]:
def best_list(date):

    best_list = []

    # url 정보 입력
    for page in range(5):
        url = f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page}&viewType=small&priceMin=&priceMax="

        # url 정보 불러오기
        response = requests.get(url)

        # html 형식으로 바꿔주기 
        html = BeautifulSoup(response.text, "html.parser")

        seed = np.random.randint(100) # 시드도 난수로 만들고
        np.random.seed(seed) # 시드 생성
        a = np.random.randint(5) # 시드에서 난수 생성
        time.sleep(a)

        for i in range(90):

            # 브랜드 명
            brand = html.select("p.item_title")[i].text.strip()

            # 상품 명
            if len(html.select("p.list_info")[i].text.replace(" ", "").split()) == 2:
                product = html.select("p.list_info")[i].text.replace(" ", "").split()[1]
                no_deli = (html.select("p.list_info")[i].text.replace(" ", "").split()[0]+" "+"예정")
            else:
                product = html.select("p.list_info")[i].text.replace(" ", "").split()[0]
                no_deli = "-"

            # 가격
            if len(html.select("p.price")[i].text.split()) == 2:
                price = html.select("p.price")[i].text.split()[0]
                sale_price = html.select("p.price")[i].text.split()[1]
            else:
                price = html.select("p.price")[i].text.split()[0]
                sale_price = "-"

            # 상품 순위, 증가 및 하락
            if len(html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()) == 3:
                rank = html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()[0]

                if "▲" in html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()[1]:
                    increase = 0
                elif "▼" in html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()[1]:
                    increase = 1

                increase_cnt = html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()[2]

            else:
                rank = html.select("p.n-label.label-default.txt_num_rank")[i].text.replace(" ", "").split()[0]
                increase = "-"
                increase_cnt = "-"


            best_dict = {
                "rank": rank,
                "brand": brand,
                "product": product,
                "price": price,
                "sale_price": sale_price,
                "increase": increase,
                "increase_cnt": increase_cnt,
                "no_deli": no_deli
            }

            best_list.append(best_dict)

    best_df = pd.DataFrame(best_list)
    best_df.to_csv(f"{date}_musinsa_best.csv", index = False)
    
    return best_df

In [ ]:
hot300(220213)

In [6]:
brand100(220213)

In [7]:
best_list(220213)